In [ ]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/CSISdefense/Fixed-price/master/Data/defense_contract_CSIScontractID_sample_100000_Unweighted.csv')

print(df.head())

In [ ]:
print(df.columns)

In [ ]:
import requests
import json

payload = {
    "filters": [
        {
            "field": "awarding_agency__toptier_agency__cgac_code",
            "operation": "equals",
            "value": "097"
        }#,
#         {
#             "field": "transaction__contract_data",
#             "operation": "is_null",
#             "value": False
#         }
    ]
}


#r = requests.post('https://api.usaspending.gov/api/v1/transactions/', json = payload)
r = requests.post('https://api.usaspending.gov/api/v1/awards/', json = payload)

In [ ]:
print(payload)

In [ ]:
type(r)

In [ ]:
r.json()
type(r.json())

In [ ]:
r.status_code

In [ ]:
r.text[:100]

In [ ]:
#api_df = pd.read_json(r.json(), lines=True)

In [ ]:
j = r.json()

#api_df = pd.DataFrame([[d['v'] for d in x['c']] for x in j['rows']],
#                  columns=[d['label'] for d in j['cols']])

In [ ]:
j['results']

In [ ]:
len(j['results'])

In [ ]:
api_df = pd.DataFrame.from_dict(j['results'])

In [ ]:
print(api_df.head())

# Transactions

Above approach had attempted retrieving on the contract level; is it possible to effectively query transactions directly?

In [55]:
import requests
import json
from retrying import retry

payload = {
    #"limit":100,
    "filters": [
        {
            "field": "awarding_agency__toptier_agency__cgac_code",
            "operation": "equals",
            "value": "097"
        }
        ,{
            "field": "transaction__contract_data",
            "operation": "is_null",
            "value": 'false'
        }
        ,{ 
            "field": ["description", "action_type_description"], 
            "operation": "contains", 
            "value": "CANCEL" 
        }
    ]
}

t = requests.post('https://api.usaspending.gov/api/v1/transactions/', json = payload)

In [48]:
print(payload)

{'limit': '100', 'filters': [{'field': 'awarding_agency__toptier_agency__cgac_code', 'operation': 'equals', 'value': '097'}, {'field': 'transaction__contract_data', 'operation': 'is_null', 'value': 'false'}, {'field': ['description', 'action_type_description'], 'operation': 'contains', 'value': 'CANCEL'}]}


In [56]:
print(len(t.json()['results']))

KeyError: 'results'

In [57]:
print(t)

<Response [500]>


In [ ]:
transactions = pd.DataFrame.from_dict(t.json()['results'])

In [ ]:
print(transactions.head())

In [ ]:
g = requests.get('https://api.usaspending.gov/api/v1/transactions/', json = payload)

print(g.json()['results'])

In [ ]:
g2 = requests.get('https://api.usaspending.gov/api/v1/transactions/?limit=250')

In [ ]:
g2.status_code

In [ ]:
g2.json()['results']

In [ ]:
get_payload = {
    'limit':250,
    'awarding_agency__toptier_agency__cgac_code':'097', #DoD top-level code
    
}

In [61]:
output = {}

for i in range(1,20):
    here = requests.get('https://api.usaspending.gov/api/v1/transactions/?limit=250&page=%s' % i,
# #                         timeout=0.5
#                         &awarding_agency__toptier_agency__cgac_code=97
                       )
#     print(len(here.json()['results']))
    print i
#    output.update(here.json())
    
#print(len(output['results']))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


KeyError: 'results'

In [115]:
LIMIT = 10000
def get_page_args(i):
    return {"url": 'https://api.usaspending.gov/api/v1/transactions/',
            "params": {"awarding_agency__toptier_agency__cgac_code=": "097", "page": i}}

# Coming up Next (1/9):

Add 'try' or 'timeout' wrapper of some sort in case one later page fails.


In [116]:
#%%timeit -n1 -r1
# faster version using requests-futures
from requests_futures.sessions import FuturesSession

session = FuturesSession(max_workers=5)
futures = [session.get('https://api.usaspending.gov/api/v1/transactions/?limit=250&page=%s' % i) for i in xrange(LIMIT)]

contracts = [(future.result().json()['results']) for future in futures]

# for future in futures:
#     print future.result().json()['results'][:2]
#     #print "/n"

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [114]:
print(type(contracts[0]))

<type 'list'>


In [97]:
print(type(futures[0].result().json().items()[:10]))

<type 'list'>


In [103]:
print((futures[0].result().json()['results'][0]))

{u'recipient': {u'parent_recipient_unique_id': u'167159198', u'domestic_or_foreign_entity_description': u'U.S. Owned Business', u'small_business_description': None, u'recipient_unique_id': u'167159198', u'legal_entity_id': 185026, u'recipient_name': u'T. SIMONS CO., INC.', u'business_categories': [u'category_business', u'hispanic_american_owned_business', u'minority_owned_business', u'woman_owned_business', u'8a_program_participant', u'self_certified_small_disadvanted_business', u'us_owned_business', u'special_designations'], u'business_types_description': u'Unknown Types', u'business_types': None, u'location': {u'address_line2': None, u'state_description': None, u'address_line1': u'230 W TEFFT ST', u'state_name': u'CALIFORNIA', u'address_line3': None, u'foreign_location_description': None, u'foreign_province': None, u'city_name': u'NIPOMO', u'location_country_code': u'USA', u'country_name': u'UNITED STATES', u'foreign_postal_code': None, u'state_code': u'CA', u'foreign_city_name': Non

In [110]:
len(contracts)

100

In [111]:
contracts[99]

[{u'action_date': u'2007-10-01',
  u'action_type': None,
  u'action_type_description': None,
  u'assistance_data': None,
  u'awarding_agency': {u'id': 4989,
   u'office_agency': None,
   u'subtier_agency': {u'abbreviation': u'PBS',
    u'name': u'Public Buildings Service',
    u'subtier_code': u'4740'},
   u'toptier_agency': {u'abbreviation': u'GSA',
    u'cgac_code': u'047',
    u'fpds_code': u'4700',
    u'name': u'General Services Administration'},
   u'toptier_flag': False},
  u'contract_data': {u'action_type_description': None,
   u'award_description': u'LEASE',
   u'cost_or_pricing_data': None,
   u'extent_compete_description': u'Full and Open Competition',
   u'fed_biz_opps_description': u'Yes',
   u'idv_type_description': None,
   u'naics': None,
   u'naics_description': None,
   u'parent_award_id': u'GS09B97097',
   u'piid': u'GS09B9709707356',
   u'product_or_service_code': u'X111',
   u'research_description': None,
   u'sam_exception_description': u'NAN',
   u'type_of_contra

In [73]:
len(futures)

1000

In [74]:
len(session)

TypeError: object of type 'FuturesSession' has no len()

In [ ]:
payload = {
    "limit":100,
    "filters": [
        {
            "field": "awarding_agency__toptier_agency__cgac_code",
            "operation": "equals",
            "value": "097"
        }#,
#         {
#             "field": "transaction__contract_data",
#             "operation": "is_null",
#             "value": False
#         }
        
        
        ,{ "field": ["description", "action_type_description"], "operation": "contains", "value": "CANCEL" }
#         ,{ "field": "description", "operation":'contains', 'value': 'cancel'}
    ]
}

# while True:
#     try:
#         t = requests.post('https://api.usaspending.gov/api/v1/transactions/', json = payload)
#         if (t.status_code == 200):
#             break
#     except Exception as inst:
#         print inst


# @retry        
# def check_it():    
#     output = requests.post('https://api.usaspending.gov/api/v1/transactions/', json = payload)
#     if (t.status_code == 200):
#         return output
#     else:
#         raise IOError("Probably a 504")

# t = check_it()